In [2]:
! ls

analysis.ipynb
anli_none_dev_r1_bigscience.T0
anli_none_dev_r1_bigscience.T0_3B
anli_none_dev_r2_bigscience.T0
anli_none_dev_r2_bigscience.T0_3B
anli_none_dev_r3_bigscience.T0
anli_none_dev_r3_bigscience.T0_3B
hellaswag_none_validation_bigscience.T0
hellaswag_none_validation_bigscience.T0_3B
super_glue_cb_validation_bigscience.T0
super_glue_cb_validation_bigscience.T0_3B
super_glue_copa_validation_bigscience.T0
super_glue_copa_validation_bigscience.T0_3B
super_glue_rte_validation_bigscience.T0
super_glue_rte_validation_bigscience.T0_3B
super_glue_wic_validation_bigscience.T0
super_glue_wic_validation_bigscience.T0_3B
super_glue_wsc.fixed_validation_bigscience.T0
super_glue_wsc.fixed_validation_bigscience.T0_3B
winogrande_winogrande_xl_validation_bigscience.T0
winogrande_winogrande_xl_validation_bigscience.T0_3B


In [3]:
from promptsource.templates import DatasetTemplates
import datasets
data = datasets.load_dataset("anli")
data = data["dev_r1"]
prompts = DatasetTemplates("anli", None)
prompt_names = prompts.all_template_names
print(prompt_names)
pname = 'guaranteed/possible/impossible'
k = 1
input, output = prompts[pname].apply(data[k])
targets = prompts[pname].get_answer_choices_list(data[k])

print(output.strip())
print("========")
print(data[k])
print("===========")
print(input)
print("========")
print(output)
print()
print(targets)

Reusing dataset anli (/data/home/chuntinz/.cache/huggingface/datasets/anli/plain_text/0.1.0/aabce88453b06dff21c201855ea83283bab0390bff746deadb30b65695755c0b)


  0%|          | 0/9 [00:00<?, ?it/s]

['GPT-3 style', 'MNLI crowdsource', 'always/sometimes/never', 'based on the previous passage', 'can we infer', 'claim true/false/inconclusive', 'consider always/sometimes/never', 'does it follow that', 'does this imply', 'guaranteed true', 'guaranteed/possible/impossible', 'justified in saying', 'must be true', 'should assume', 'take the following as truth']
Guaranteed
{'uid': 'e66af435-eef1-491b-af2a-4825d54611e1', 'premise': 'Franco Zeffirelli, KBE Grande Ufficiale OMRI (] ; born 12 February 1923) is an Italian director and producer of operas, films and television. He is also a former senator (1994–2001) for the Italian centre-right "Forza Italia" party. Recently, Italian researchers have found that he is one of the few distant relatives of Leonardo da Vinci.', 'hypothesis': 'Franco Zeffirelli had a political career', 'label': 0, 'reason': 'Franco Zeffirelli was a senator so he had a political career. The system likely was fooled because I used words not used in the context.'}
Assume

In [2]:
import numpy as np
from collections import Counter
import scipy.stats

def read_file(fname):
    lidx = -1
    eidx = -1
    all_preds_per_example = []
    all_golds = []
    all_preds_per_prompt = []
    with open(fname) as fin:
        for line in fin:
            lidx += 1
            if lidx == 0:
                fields = line.strip().split(',')
                num_prompts, num_examples = int(fields[0].split("=")[1].strip()), int(fields[1].split("=")[1].strip())
                all_preds_per_prompt = [[] for _ in range(num_prompts)]
            elif lidx == 1:
                fields = line.strip().split(',')
                max_acc, median_acc, min_acc, ensemble_acc = float(fields[0].split("=")[1].strip()), float(fields[1].split("=")[1].strip()), float(fields[3].split("=")[1].strip()), float(fields[-1].split("=")[1].strip())
            elif lidx == 2:
                prompts_accuracy = list(map(float, line.strip().split(':')[1].strip().split()))
            elif lidx == 3:
                pass
            elif line.startswith("====="):
                eidx += 1
            elif line.startswith("eid"):
                fields = line.strip().split(',')
                gold = int(fields[1].split("=")[1])
                preds = list(map(int, fields[-1].strip().split()))
                all_preds_per_example.append(preds)
                all_golds.append(gold)
                for pid, pp in enumerate(preds):
                    all_preds_per_prompt[pid].append(pp)
            else:
                # read pred probs
                pass
    return num_prompts, num_examples, all_preds_per_example, all_preds_per_prompt, all_golds, prompts_accuracy, max_acc, median_acc, min_acc, ensemble_acc

test_file_names = ['anli_none_dev_r1_bigscience.T0', 'anli_none_dev_r2_bigscience.T0', 'anli_none_dev_r3_bigscience.T0', 
                  'hellaswag_none_validation_bigscience.T0', 'super_glue_cb_validation_bigscience.T0',
                  'super_glue_copa_validation_bigscience.T0', 'super_glue_rte_validation_bigscience.T0',
                  'super_glue_wic_validation_bigscience.T0', 'super_glue_wsc.fixed_validation_bigscience.T0', 'winogrande_winogrande_xl_validation_bigscience.T0']

t0_11b = {}
for fname in test_file_names:
    num_prompts, num_examples, all_preds_per_example, all_preds_per_prompt, all_golds, prompts_accuracy, max_acc, median_acc, min_acc, ensemble_acc = read_file(fname)
    t0_11b[fname] = {"nprompts": num_prompts, 'nexamples': num_examples, 'all_preds_per_example': all_preds_per_example, 'all_preds_per_prompt': all_preds_per_prompt, "all_golds": all_golds, 
                    "prompts_accuracy": prompts_accuracy, 'max_acc': max_acc, 'median_acc': median_acc, 'min_acc': min_acc, 'ensemble_acc': ensemble_acc}
    
t0_3b = {}
for fname in test_file_names:
    num_prompts, num_examples, all_preds_per_example, all_preds_per_prompt, all_golds, prompts_accuracy, max_acc, median_acc, min_acc, ensemble_acc = read_file(fname+"_3B")
    t0_3b[fname] = {"nprompts": num_prompts, 'nexamples': num_examples, 'all_preds_per_example': all_preds_per_example, 'all_preds_per_prompt': all_preds_per_prompt, "all_golds": all_golds, 
                    "prompts_accuracy": prompts_accuracy, 'max_acc': max_acc, 'median_acc': median_acc, 'min_acc': min_acc, 'ensemble_acc': ensemble_acc}
    

def cal_task_prompt_oracle_acc(task_dict, suffix=""):
    for fname in test_file_names:
        results = task_dict[fname]
        degenerate_prompts = set()
        for pid, preds in enumerate(task_dict[fname]['all_preds_per_prompt']):
            if len(set(preds)) == 1:
                degenerate_prompts.add(pid)
            else:
                count = Counter(preds)
                if max(np.array(list(count.values()))*1.0 / sum(count.values())) > 0.8:
                    degenerate_prompts.add(pid)
            
        all_preds_per_example = task_dict[fname]['all_preds_per_example']
        all_golds = task_dict[fname]['all_golds']
        print(degenerate_prompts)
        count = 0
        for pred_per_example, gold in zip(all_preds_per_example, all_golds):
            pred_per_example = [pred_per_example[i] for i in range(len(all_preds_per_example[0])) if i not in degenerate_prompts]
            count += 1 if gold in pred_per_example else 0
#             if pred_per_example.count(gold) >= 2:
#                 count += 1
        oracle_acc = count * 100. / len(all_golds)
        max_acc, median_acc, min_acc, ensemble_acc = task_dict[fname]['max_acc'], task_dict[fname]['median_acc'], task_dict[fname]['min_acc'], task_dict[fname]['ensemble_acc'], 
        print("{}{}: max_acc = {}, oracle_acc = {}, median_acc = {}, min_acc = {}, ensemble_acc = {}".format(fname, suffix, max_acc, oracle_acc, median_acc, min_acc, ensemble_acc))
        
cal_task_prompt_oracle_acc(t0_11b)
cal_task_prompt_oracle_acc(t0_3b, "_3B")

def cal_correlation():
    t11b_accs, t3b_accs = [], []
    for fname in test_file_names:
        t11b_accs.extend(t0_11b[fname]["prompts_accuracy"])
        t3b_accs.extend(t0_3b[fname]["prompts_accuracy"])
    corr = scipy.stats.pearsonr(t11b_accs, t3b_accs)
    print("correlation = {}".format(corr))

cal_correlation()

{10}
anli_none_dev_r1_bigscience.T0: max_acc = 46.5, oracle_acc = 80.2, median_acc = 44.7, min_acc = 33.3, ensemble_acc = 45.5
{10}
anli_none_dev_r2_bigscience.T0: max_acc = 40.8, oracle_acc = 78.9, median_acc = 39.7, min_acc = 33.5, ensemble_acc = 39.9
{10}
anli_none_dev_r3_bigscience.T0: max_acc = 44.25, oracle_acc = 79.58333333333333, median_acc = 42.67, min_acc = 33.92, ensemble_acc = 43.33
set()
hellaswag_none_validation_bigscience.T0: max_acc = 34.13, oracle_acc = 64.35968930491934, median_acc = 33.67, min_acc = 33.18, ensemble_acc = 34.8
{10}
super_glue_cb_validation_bigscience.T0: max_acc = 80.36, oracle_acc = 98.21428571428571, median_acc = 78.57, min_acc = 8.93, ensemble_acc = 78.57
set()
super_glue_copa_validation_bigscience.T0: max_acc = 96.0, oracle_acc = 99.0, median_acc = 93.0, min_acc = 82.0, ensemble_acc = 94.0
set()
super_glue_rte_validation_bigscience.T0: max_acc = 84.84, oracle_acc = 93.14079422382672, median_acc = 82.31, min_acc = 73.65, ensemble_acc = 84.48
{1, 3,

In [10]:
! export TRANSFORMERS_CACHE=../pretrain_models/huggingface
! export HF_DATASETS_CACHE=../pretrain_models/huggingface
! export HF_METRICS_CACHE=../pretrain_models/huggingface
! cache_dir=../pretrain_models/huggingface

# T0 train set

from os import read
from promptsource.templates import DatasetTemplates
import datasets
cache_dir='../pretrain_models/huggingface'


datasets_with_more_than_2_original_prompts = 0
datasets_with_only_nonoriginal_prompts = 0

def read_prompts(dataset, subset, split):
    global datasets_with_more_than_2_original_prompts
    global datasets_with_only_nonoriginal_prompts
    print()
    print("==="*100)
    print(dataset, subset, split)
    data = datasets.load_dataset(dataset, subset, cache_dir=cache_dir)
    data = data[split]
    prompts = DatasetTemplates(dataset, subset)
    prompt_names = prompts.all_template_names
    original_prompts =[n for n in prompt_names if prompts[n].metadata.original_task]
    non_original_prompts = [n for n in prompt_names if not prompts[n].metadata.original_task]
    print("total prompts = {}, original prompts = {}".format(len(prompt_names), len(original_prompts)))

    choices = prompts[prompt_names[0]].get_answer_choices_list(data[0])
    print(data[0].keys())
    if choices is not None:
        print("number of choices = {}".format(len(choices)))
    else:
        print(">>>>>>>>>>>>>>>>>>>>>>>>>>>> no choices >>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    if len(original_prompts) > 0:
        print(prompts[original_prompts[0]].apply(data[0]))
    if len(non_original_prompts) > 0:
        cc = prompts[non_original_prompts[0]].get_answer_choices_list(data[0])
        if cc is not None:
            print(len(cc))
        print("=====================")
        print(prompts[non_original_prompts[0]].jinja)
        print("=====================")
        print(prompts[non_original_prompts[0]].apply(data[0]))
    
    if len(original_prompts) >= 2:
        datasets_with_more_than_2_original_prompts += 1
    if len(original_prompts) == 0:
        datasets_with_only_nonoriginal_prompts += 1

# Paraphrase Identification glue/mrpc
read_prompts("glue", "mrpc", "validation")
# Paraphrase Identification glue/qqp
read_prompts("glue", 'qqp', "validation")
# Paraphrase Identification paws/labeled_final
read_prompts("paws", 'labeled_final', "validation")

# Closed-Book QA ai2_arc/ARC_Challenge
read_prompts("ai2_arc", 'ARC-Challenge', "validation")
# Closed-Book QA ai2_arc/ARC_Easy
read_prompts("ai2_arc", 'ARC-Easy', "validation")
# Closed-Book QA kilt_tasks/hotpotqa
read_prompts("kilt_tasks", 'hotpotqa', "validation")
# Closed-Book QA trivia_qa/unfiltered
read_prompts("trivia_qa", 'unfiltered', "validation")
# Closed-Book QA web_questions
read_prompts("web_questions", None, "test")
# Closed-Book QA wiki_qa
read_prompts("wiki_qa", None, "validation")

# Extractive QA adversarial_qa/dbidaf
read_prompts("adversarial_qa", 'dbidaf', "validation")
# Extractive QA adversarial_qa/dbert
read_prompts("adversarial_qa", 'dbert', "validation")
# Extractive QA adversarial_qa/droberta
read_prompts("adversarial_qa", 'droberta', "validation")
# Extractive QA duorc/SelfRC
read_prompts("duorc", 'SelfRC', "validation")
# Extractive QA duorc/ParaphraseRC
read_prompts("duorc", 'ParaphraseRC', "validation")
# Extractive QA ropes
read_prompts("ropes", None, "validation")
# Extractive QA squad_v2
read_prompts("squad_v2", None, "validation")
# Extractive QA super_glue/record
read_prompts("super_glue", 'record', "validation")
# Extractive QA quoref
read_prompts("quoref", None, "validation")
# Extractive QA tydiqa
read_prompts("tydiqa", "primary_task", "validation")

# Multiple-Choice QA cos_e/v1.11
read_prompts("cos_e", 'v1.11', "validation")
# Multiple-Choice QA cosmos_qa
read_prompts("cosmos_qa", None, "validation")
# Multiple-Choice QA dream
read_prompts("dream", None, "validation")
# Multiple-Choice QA openbookqa/main 
read_prompts("openbookqa", 'main', "validation")
# Multiple-Choice QA qasc
read_prompts("qasc", None, "validation")
# Multiple-Choice QA quail 
read_prompts("quail", None, "validation")
# Multiple-Choice QA quarel 
read_prompts("quarel", None, "validation")
# Multiple-Choice QA quartz
read_prompts("quartz", None, "validation")
# Multiple-Choice QA race/high
read_prompts("race", "high", "validation")
# Multiple-Choice QA race/middle
read_prompts("race", "middle", "validation")
# Multiple-Choice QA sciq
read_prompts("sciq", None, "validation")
# Multiple-Choice QA social_i_qa
read_prompts("social_i_qa", None, "validation")
# Multiple-Choice QA super_glue/boolq
read_prompts("super_glue", 'boolq', "validation")
# Multiple-Choice QA super_glue/multirc
read_prompts("super_glue", 'multirc', "validation")
# Multiple-Choice QA wiki_hop/original
read_prompts("wiki_hop", 'original', "validation")
# Multiple-Choice QA wiqa 
read_prompts("wiqa", None, "validation")
# Multiple-Choice QA piqa
read_prompts("piqa", None, "validation")

# Sentiment amazon_polarity
read_prompts("amazon_polarity", None, "test")
# Sentiment app_reviews
read_prompts("app_reviews", None, "train")
# Sentiment imdb
read_prompts("imdb", None, "test")
# Sentiment rotten_tomatoes
read_prompts("rotten_tomatoes", None, "validation")
# Sentiment yelp_review_full
read_prompts("yelp_review_full", None, "test")

# Structure-to-Text common_gen
read_prompts("common_gen", None, "validation")
# Structure-to-Text wiki_bio
read_prompts("wiki_bio", None, "val")

# Summarization cnn_dailymail/3.0.0
read_prompts("cnn_dailymail", '3.0.0', "validation")
# Summarization gigaword
read_prompts("gigaword", None, "validation")
# Summarization multi_news
read_prompts("multi_news", None, "validation")
# Summarization xsum
read_prompts("xsum", None, "validation")
# Summarization samsum
read_prompts("samsum", None, "validation")

# Topic Classification ag_news
read_prompts("ag_news", None, "test")
# Topic Classification dbpedia_14
read_prompts("dbpedia_14", None, "test")
# Topic Classification trec
read_prompts("trec", None, "test")

print("datasets with more than 2 original prompts: {}".format(datasets_with_more_than_2_original_prompts))
print("datasets with only non-original prompts: {}".format(datasets_with_only_nonoriginal_prompts))


glue mrpc validation


Reusing dataset glue (../pretrain_models/huggingface/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

total prompts = 7, original prompts = 5
dict_keys(['sentence1', 'sentence2', 'label', 'idx'])
number of choices = 2
['Are the following two sentences "equivalent" or "not equivalent"?\nHe said the foodservice pie business doesn \'t fit the company \'s long-term growth strategy .\n" The foodservice pie business does not fit our long-term growth strategy .', 'equivalent']
{% if label == 1 %}
Paraphrase the following sentence: {{sentence1}}
|||
{{sentence2}}
{% endif %}
["Paraphrase the following sentence: He said the foodservice pie business doesn 't fit the company 's long-term growth strategy .", '" The foodservice pie business does not fit our long-term growth strategy .']

glue qqp validation


Reusing dataset glue (../pretrain_models/huggingface/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


...

  0%|          | 0/2 [00:00<?, ?it/s]

total prompts = 4, original prompts = 4
dict_keys(['label', 'title', 'content'])
number of choices = 14
["TY KU -  TY KU /taɪkuː/ is an American alcoholic beverage company that specializes in sake and other spirits. The privately-held company was founded in 2004 and is headquartered in New York City New York. While based in New York TY KU's beverages are made in Japan through a joint venture with two sake breweries. Since 2011 TY KU's growth has extended its products into all 50 states. Given a choice of categories company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film or written work, the text refers to which one?", 'Company']

trec None test


Using custom data configuration default
Reusing dataset trec (../pretrain_models/huggingface/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9)


  0%|          | 0/2 [00:00<?, ?it/s]

total prompts = 18, original prompts = 7
dict_keys(['label-coarse', 'label-fine', 'text'])
number of choices = 2
['What is this question asking for?\n\nHow far is it from Denver to Aspen ?', '']
2
{% set label_mapping={34:0, 3:1} %} 
{% if label_coarse == 2 %}
Is this question asking for an {{', '.join(answer_choices)}}?
{{text}}
|||
{{answer_choices[label_mapping[label_fine]] }}
{% endif %}
['']
datasets with more than 2 original prompts: 47
datasets with only non-original prompts: 3


In [1]:
! export TRANSFORMERS_CACHE=../pretrain_models/huggingface
! export HF_DATASETS_CACHE=../pretrain_models/huggingface
! export HF_METRICS_CACHE=../pretrain_models/huggingface
! cache_dir=../pretrain_models/huggingface

# T0 train set

from os import read
from promptsource.templates import DatasetTemplates
import datasets
cache_dir='../pretrain_models/huggingface'

def read_prompts(dataset, subset, split):
    print()
    print("==="*100)
    print(dataset, subset, split)
    data = datasets.load_dataset(dataset, subset, cache_dir=cache_dir)
    data = data[split]
    prompts = DatasetTemplates(dataset, subset)
    prompt_names = prompts.all_template_names
    print(len(prompt_names))

    choices = prompts[prompt_names[0]].get_answer_choices_list(data[0])
    if choices is not None:
        print(data[0].keys())
        print(prompts[prompt_names[0]].jinja)
        print("number of choices = {}".format(len(choices)))
    else:
        print("======= no choices =========")
        print(data[0].keys())
        print(prompts[prompt_names[0]].jinja)
        print(prompts[prompt_names[0]].apply(data[0]))
       
# read_prompts("hotpot_qa", "fullwiki", "validation")

data = datasets.load_dataset('bigscience/P3', cache_dir=cache_dir)

2022-09-19 18:50:27.308876: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-19 18:50:31.851941: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


ImportError: To be able to use bigscience/P3, you need to install the following dependency: tensorflow.
Please install it using 'pip install tensorflow' for instance'